In [2]:
import pandas as pd

**Cleaning data**

In [3]:
dataset = pd.read_csv('Coup_Data_v2.0.0.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [8]:
# убираем уже не существующие страны
clean_data = dataset[dataset.country != 'USSR']

In [17]:
# учёт разных названий одной и той же страны
clean_data = clean_data.replace(to_replace={'Kyrgyz Republic':'Kyrgyzstan', "Cote d'Ivoire":'Ivory Coast','Congo':'Democratic Republic of the Congo'})
#clean_data.iloc[279] проверить что в конкретной строке

In [20]:
# запись фрейма в файл
clean_data.to_csv('Coup_Data_v2.0.0_clean.csv', encoding='utf-8')

**Computing Coups for each country**

In [21]:
data = pd.read_csv('Coup_Data_v2.0.0_clean.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [23]:
# госперевороты для заданного периода
start_date = 1989
data_filtered = data[data['year'] > start_date]

In [24]:
# сортировка стран по алфавиту
data_country_filtered = data_filtered.sort_values('country', ascending=True)
# список всех стран в алфавитном порядке
country_list = data_country_filtered['country'].unique()
# количество стран в датасете
country_num = len(country_list)
print(country_num)

83


In [25]:
# госперевороты для заданной страны
len(data[data['country'] == 'Russia'])

2

In [31]:
# частотность переворотов за последние 50 лет
# количество переворотов == количество строчек с этой страной
data_country_numbered = data_filtered.groupby('country').count()
coup_numbers = list(data_country_numbered['coup_id'])

In [32]:
# таблица для записи итоговых данных
result_data = pd.DataFrame({'country':country_list, 'coup_num':coup_numbers}, columns=['country', 'coup_num'])

In [33]:
# статистика по столбцу
result_data['coup_num'].describe()
# среднее значение
result_data['coup_num'].mean()

3.0963855421686746

In [57]:
# измение имени столбца
result_data.rename(columns={'Unnamed: 0':'id'}, inplace = True)

In [60]:
result_data.columns

Index(['country', 'coup_num', 'p_month', 'p_year', 'p_year_smooth',
       'p_month_smooth'],
      dtype='object')

**Computing Probabilities**

**Вычисление частот и сглаженных частот**

пусть средняя частота переворотов для всех стран - m
пусть коэффициент сглаживания a « 1 (например, 0.1)

путь частота переворотов для конкретной страны mZ

p_smooth = (1-a)mZ + am

In [38]:
a = 0.1
p_month = list()
p_year = list()
p_year_smooth = list()
p_month_smooth = list()

In [39]:
# частоты по месяцам
for num in result_data['coup_num']:
   p_month.append(num/600*100) 
# частоты по годам
for num in result_data['coup_num']:
   p_year.append(num/50*100) 

In [40]:
# запись значение в фрейм
result_data['p_month'] = p_month
result_data['p_year'] = p_year

In [41]:
# сглаженные частоты по месяцам
m = result_data['p_month'].mean()
for p in result_data['p_month']:
   p_month_smooth.append(round(((1-a)*p + a*m), 3)) 
# сглаженные частоты по годам
m = result_data['p_year'].mean()
for p in result_data['p_year']:
   p_year_smooth.append(round(((1-a)*p + a*m))) 

In [42]:
# запись значение в фрейм
result_data['p_year_smooth'] = p_year_smooth
result_data['p_month_smooth'] = p_month_smooth

In [52]:
# чтение списка стран из файла
# !!!! change path !!!
countries = pd.read_csv('/home/anastasia/PycharmProjects/coup-prediction/data/country_names.csv', encoding='utf-8', delimiter=',', error_bad_lines=False) 

In [61]:
result_data_with_un = result_data.copy()
# добавляем в список стран, которых не было в Coup d'etait
list_for_adding = list(set(countries['country_name']) - set(result_data['country']))        
# частоты для них проставляем средние
p_month_smooth_average = result_data['p_month_smooth'].mean()
p_month_average = result_data['p_month'].mean()
p_year_smooth_average = result_data['p_year_smooth'].mean()
p_year_average = result_data['p_year'].mean()
coup_average = result_data['coup_num'].mean()


In [62]:
data_un = pd.DataFrame({'country':list_for_adding, 'coup_num':coup_average, 'p_month': p_month_average,'p_year':p_year_average, 
                        'p_year_smooth':p_year_smooth_average, 'p_month_smooth': p_month_smooth_average}, 
                       columns=['country', 'coup_num', 'p_month', 'p_year', 'p_year_smooth', 'p_month_smooth'])

In [63]:
data_un

,country,coup_num,p_month,p_year,p_year_smooth,p_month_smooth
0,Slovenia,3.096386,0.516064,6.192771,6.012048,0.516458
1,Vanuatu,3.096386,0.516064,6.192771,6.012048,0.516458
2,Switzerland,3.096386,0.516064,6.192771,6.012048,0.516458
3,Vietnam,3.096386,0.516064,6.192771,6.012048,0.516458
4,Ghana,3.096386,0.516064,6.192771,6.012048,0.516458
...,...,...,...,...,...,...
109,El Salvador,3.096386,0.516064,6.192771,6.012048,0.516458
110,Estonia,3.096386,0.516064,6.192771,6.012048,0.516458
111,United Arab Emirates,3.096386,0.516064,6.192771,6.012048,0.516458
112,Uruguay,3.096386,0.516064,6.192771,6.012048,0.516458


In [64]:
result_data_with_un.append(data_un, ignore_index=True)

,country,coup_num,p_month,p_year,p_year_smooth,p_month_smooth
0,Afghanistan,4.000000,0.666667,8.000000,8.000000,0.652000
1,Albania,1.000000,0.166667,2.000000,2.000000,0.202000
2,Algeria,5.000000,0.833333,10.000000,10.000000,0.802000
3,Argentina,1.000000,0.166667,2.000000,2.000000,0.202000
4,Armenia,1.000000,0.166667,2.000000,2.000000,0.202000
...,...,...,...,...,...,...
192,El Salvador,3.096386,0.516064,6.192771,6.012048,0.516458
193,Estonia,3.096386,0.516064,6.192771,6.012048,0.516458
194,United Arab Emirates,3.096386,0.516064,6.192771,6.012048,0.516458
195,Uruguay,3.096386,0.516064,6.192771,6.012048,0.516458


In [65]:
# запись фрейма в файл
result_data_with_un.to_csv('probas_for_coup_detait_with_un.csv', encoding='utf-8')